Jupyter Notebook for dataset preprocessing for 'dreamsss' dataset. 

In [4]:
#Import statements
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import math
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, cross_val_score

In [5]:
dreams = pd.read_csv('dreams_dataset.csv')
print(dreams.shape)
dreams.head(10)

(21000, 21)


,dream_id,dreamer,description,dream_date,dream_language,text_dream,characters_code,emotions_code,aggression_code,friendliness_code,...,Male,Animal,Friends,Family,Dead&Imaginary,Aggression/Friendliness,A/CIndex,F/CIndex,S/CIndex,NegativeEmotions
0,1,alta,Alta: a detailed dreamer,1957,en,"The one at the Meads's house, where it's bigge...","2ISA, 1MKA, 1FDA, 1IOA, 2ISA",NaN,2IKA > Q,2IKA 4> Q,...,0.500000,0.000000,0.200000,0.200000,0.0,0.000000,0.200000,0.200000,0.0,0.0
1,2,alta,Alta: a detailed dreamer,8/11/1967,en,I'm at a family reunion in a large fine house ...,"2ISA, people, 2ISA",SD 2IKA,"D > Q, Q > 2ISA",NaN,...,0.000000,0.000000,0.000000,0.000000,0.0,1.000000,0.666667,0.000000,0.0,1.0
2,3,alta,Alta: a detailed dreamer,8/1/1985,en,I watch a plane fly past and shortly realize i...,"2ISA, 2ISA, 1FSA, 1MBA, 1IOA, 2ISA, 2FDA","SD 1ISA, AP D, AP D","It PRP >, It PRP >, D > 1FKA",NaN,...,0.333333,0.000000,0.000000,0.285714,0.0,1.000000,0.428571,0.000000,0.0,1.0
3,4,alta,Alta: a detailed dreamer,1985?,en,Me pulling the green leaves and berries off so...,"1MAA, 1FMA, 2ISA, 2IKA, 1ANI, 1ANI, 1IOA, 2ISA...","SD 2ISA, SD D","Q > Q, 2ISA > Q, 2ISA > Q, D > 1MSA","1IKA 4> Q, 2ISA 4> 2ISA",...,0.666667,0.176471,0.142857,0.142857,0.0,1.000000,0.235294,0.117647,0.0,1.0
4,5,alta,Alta: a detailed dreamer,1985?,en,I'm in a room that reminds me of (but definite...,"1IRA, 1MSA, 1ISA, 2ISA, 1ISA, 1IKA","AP D, AP D, AP 1MSA, CO D, SD D, AP D","1MSA > D, Q > Q, D > 2IKA, D > 2IKA, D > 1MSA,...",D 4> Q,...,1.000000,0.000000,0.166667,0.166667,0.0,0.875000,1.333333,0.166667,0.0,1.0
5,6,alta,Alta: a detailed dreamer,1985?,en,Living next door to Loretta in an apartment - ...,"1FSA, 2ISA, 1FDA, 1IIC",HA 1FSA,D > 2IKA,NaN,...,0.000000,0.000000,0.000000,0.500000,0.0,1.000000,0.250000,0.000000,0.0,0.0
6,7,alta,Alta: a detailed dreamer,1985?,en,Kidnapped - I'm on my way somewhere else (by c...,"2ISA, 2IOA, 1IOA, 1ISA, 2IKA, 2ISA, 2ISA","AN 1ISA, AN D, SD 2ISA, AN D","Q > 2IKA, D > 2ISA, D > Q, 2IKA > D, D > Q, 1I...",NaN,...,0.000000,0.000000,0.142857,0.000000,0.0,1.000000,1.571429,0.000000,0.0,1.0
7,8,alta,Alta: a detailed dreamer,1985?,en,"I'm alone in an apartment - old place, inside ...","2ISA, 1ISA, 1ISA","AP D, SD D","D > D, 1MSA > D, 1MSA > D, D > 1MSA, D > D, D ...",D 4> D,...,0.000000,0.000000,0.000000,0.000000,0.0,0.888889,2.666667,0.333333,0.0,1.0
8,9,alta,Alta: a detailed dreamer,1985?,en,I'm somewhere with a friend I don't want to be...,"1IKA, stepfather, 1ISB, 1MSA",NaN,D > Q,NaN,...,1.000000,0.000000,0.250000,0.000000,0.0,1.000000,0.250000,0.000000,0.0,0.0
9,10,alta,Alta: a detailed dreamer,1985?,en,"I am in some public place, reminds me of the g...","2ISA, 1MKA",NaN,NaN,NaN,...,1.000000,0.000000,0.500000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0


In [15]:
#Add an additional column 'Gender' for the gender of the dreamser, place it in index 2
dreams.insert(2, 'gender', 'n/a', allow_duplicates=True)
dreams.head(5)


,dream_id,dreamer,gender,gender,description,dream_date,dream_language,text_dream,characters_code,emotions_code,...,Male,Animal,Friends,Family,Dead&Imaginary,Aggression/Friendliness,A/CIndex,F/CIndex,S/CIndex,NegativeEmotions
0,1,alta,n/a,True,Alta: a detailed dreamer,1957,en,"The one at the Meads's house, where it's bigge...","2ISA, 1MKA, 1FDA, 1IOA, 2ISA",NaN,...,0.500000,0.000000,0.200000,0.200000,0.0,0.000,0.200000,0.200000,0.0,0.0
1,2,alta,n/a,True,Alta: a detailed dreamer,8/11/1967,en,I'm at a family reunion in a large fine house ...,"2ISA, people, 2ISA",SD 2IKA,...,0.000000,0.000000,0.000000,0.000000,0.0,1.000,0.666667,0.000000,0.0,1.0
2,3,alta,n/a,True,Alta: a detailed dreamer,8/1/1985,en,I watch a plane fly past and shortly realize i...,"2ISA, 2ISA, 1FSA, 1MBA, 1IOA, 2ISA, 2FDA","SD 1ISA, AP D, AP D",...,0.333333,0.000000,0.000000,0.285714,0.0,1.000,0.428571,0.000000,0.0,1.0
3,4,alta,n/a,True,Alta: a detailed dreamer,1985?,en,Me pulling the green leaves and berries off so...,"1MAA, 1FMA, 2ISA, 2IKA, 1ANI, 1ANI, 1IOA, 2ISA...","SD 2ISA, SD D",...,0.666667,0.176471,0.142857,0.142857,0.0,1.000,0.235294,0.117647,0.0,1.0
4,5,alta,n/a,True,Alta: a detailed dreamer,1985?,en,I'm in a room that reminds me of (but definite...,"1IRA, 1MSA, 1ISA, 2ISA, 1ISA, 1IKA","AP D, AP D, AP 1MSA, CO D, SD D, AP D",...,1.000000,0.000000,0.166667,0.166667,0.0,0.875,1.333333,0.166667,0.0,1.0


In [4]:
#Split train & test set with 80:20 ratio 

#X = df.iloc[:, :-1]
#y = df.iloc[:, -1]

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [5]:
#Remove unnecessary columns
#dreams.drop('dreams_language', axis=1, inplace=True)
#dreams.head(10)

In [6]:
dreams.isna().sum()

 dream_id                      0
dreamer                        0
description                    0
dream_date                     0
dream_language                 0
text_dream                     0
characters_code              468
emotions_code               9914
aggression_code             5568
friendliness_code           9761
sexuality_code             14530
Male                           0
Animal                         0
Friends                        0
Family                         0
Dead&Imaginary                 0
Aggression/Friendliness        0
A/CIndex                       0
F/CIndex                       0
S/CIndex                       0
NegativeEmotions               0
dtype: int64

In [17]:
print(dreams.Male.min(), dreams.Male.max())
print(dreams.Animal.min(), dreams.Animal.max())
print(dreams.Friends.min(), dreams.Friends.max())
print(dreams.Family.min(), dreams.Family.max())
print(dreams['Dead&Imaginary'].min(), dreams['Dead&Imaginary'].max())
print(dreams['Aggression/Friendliness'].min(), dreams['Aggression/Friendliness'].max())
print(dreams['A/CIndex'].min(), dreams['A/CIndex'].max())
print(dreams['F/CIndex'].min(), dreams['F/CIndex'].max())
print(dreams['S/CIndex'].min(), dreams['S/CIndex'].max())
print(dreams.NegativeEmotions.min(), dreams.NegativeEmotions.max())

0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 10.0
0.0 11.0
0.0 8.0
0.0 1.0


In [43]:
print(np.where(dreams['A/CIndex'] > 1.0))
dreams.query('`A/CIndex` > 1.0')

(array([    4,     6,     7, ..., 20987, 20993, 20998], dtype=int64),)


,dream_id,dreamer,description,dream_date,dream_language,text_dream,characters_code,emotions_code,aggression_code,friendliness_code,...,Male,Animal,Friends,Family,Dead&Imaginary,Aggression/Friendliness,A/CIndex,F/CIndex,S/CIndex,NegativeEmotions
4,5,alta,Alta: a detailed dreamer,1985?,en,I'm in a room that reminds me of (but definite...,"1IRA, 1MSA, 1ISA, 2ISA, 1ISA, 1IKA","AP D, AP D, AP 1MSA, CO D, SD D, AP D","1MSA > D, Q > Q, D > 2IKA, D > 2IKA, D > 1MSA,...",D 4> Q,...,1.0,0.000,0.166667,0.166667,0.0,0.875000,1.333333,0.166667,0.000000,1.0
6,7,alta,Alta: a detailed dreamer,1985?,en,Kidnapped - I'm on my way somewhere else (by c...,"2ISA, 2IOA, 1IOA, 1ISA, 2IKA, 2ISA, 2ISA","AN 1ISA, AN D, SD 2ISA, AN D","Q > 2IKA, D > 2ISA, D > Q, 2IKA > D, D > Q, 1I...",NaN,...,0.0,0.000,0.142857,0.000000,0.0,1.000000,1.571429,0.000000,0.000000,1.0
7,8,alta,Alta: a detailed dreamer,1985?,en,"I'm alone in an apartment - old place, inside ...","2ISA, 1ISA, 1ISA","AP D, SD D","D > D, 1MSA > D, 1MSA > D, D > 1MSA, D > D, D ...",D 4> D,...,0.0,0.000,0.000000,0.000000,0.0,0.888889,2.666667,0.333333,0.000000,1.0
20,21,alta,Alta: a detailed dreamer,1/18/1986,en,I'm on a footbridge with some friends and ther...,"2IKA, 1ISB",NaN,"D > 2IKA, 2ISA > Q, 2ISA > Q",D 4> D,...,0.0,0.000,0.500000,0.000000,0.0,0.500000,1.500000,0.500000,0.000000,0.0
21,22,alta,Alta: a detailed dreamer,1/19/1986,en,At work (?) was on the elevator with an older ...,"1FDA, 2ANI, 1ISA, 2ISA, 1FSA","CO 1FSA, HA 1IKA","1FSA > 1FSA, 1FSA > D, D > Q, 2ISA > Q, 2ISA >...",NaN,...,0.0,0.200,0.000000,0.250000,0.0,1.000000,1.400000,0.000000,0.000000,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20955,33022,west_coast_teens,West Coast teenage girls,"F, age 15",en,Last night I had a dream about tornadoes. It w...,"1MKA, 2ISA",NaN,"Q > Q, D > Q, D > D, D > Q",D 4> Q,...,1.0,0.000,0.500000,0.000000,0.0,0.750000,2.000000,0.500000,0.500000,0.0
20956,33023,west_coast_teens,West Coast teenage girls,"F, age 15",en,There is a boy that is very close to me. I lov...,"1MAA, 1MKA, 2IKA",NaN,"D > 1MSA, 1MSA > D, D > 1MSA, D > Q","D 4> him, 2ISA 4> D",...,1.0,0.000,0.666667,0.333333,0.0,0.666667,1.333333,0.666667,0.333333,0.0
20987,33054,west_coast_teens,West Coast teenage girls,"F, age 17",en,I was driving a Datsun blue truck. Me and my f...,"1IKA, 1MSA, 1ISA, 1ISA, 1ISB, lion, 1IOA, 1ANI","AP 2ISA, SD 1MSA, AP D","D > 1IKB, It PRP >, Q > 1ANI, D > 1ANI, D > 1A...",1IKA 4> 1IKB,...,1.0,0.125,0.142857,0.000000,0.0,1.000000,1.250000,0.125000,0.000000,1.0
20993,33060,west_coast_teens,West Coast teenage girls,"F, age 18",en,I was being chased by two people. It was a man...,"2ISA, woman",AP 1MSA,"D > 2IKA, 2ISA > D, D > Q, 1MKA > Q, D > 1MSA",NaN,...,0.0,0.000,0.000000,0.000000,0.0,1.000000,2.500000,0.000000,0.000000,1.0


In [44]:
print(np.where(dreams['F/CIndex'] > 1.0))
dreams.query('`F/CIndex` > 1.0')

(array([   93,   305,   352,   395,   431,   446,   582,   608,   660,
         675,   688,   706,   708,   736,   753,   758,   766,   774,
         785,   794,   801,   802,   804,   813,   818,   835,   867,
         893,   896,   927,   935,   945,   960,  1002,  1017,  1041,
        1076,  1078,  1097,  1113,  1116,  1119,  1125,  1137,  1162,
        1199,  1209,  1211,  1216,  1239,  1244,  1283,  1303,  1336,
        1343,  1368,  1369,  1397,  1410,  1416,  1422,  1433,  1465,
        1466,  1474,  1482,  1489,  1492,  1502,  1556,  1563,  1606,
        1607,  1613,  1623,  1645,  1646,  1648,  1677,  1678,  1709,
        1711,  1720,  1734,  1743,  1754,  1755,  1805,  1820,  1862,
        1866,  1875,  1884,  1898,  1899,  1905,  1921,  1950,  1959,
        1960,  1987,  2025,  2029,  2034,  2048,  2061,  2062,  2077,
        2088,  2104,  2120,  2126,  2133,  2141,  2153,  2167,  2199,
        2200,  2205,  2214,  2231,  2242,  2253,  2266,  2282,  2295,
        2308,  2309

,dream_id,dreamer,description,dream_date,dream_language,text_dream,characters_code,emotions_code,aggression_code,friendliness_code,...,Male,Animal,Friends,Family,Dead&Imaginary,Aggression/Friendliness,A/CIndex,F/CIndex,S/CIndex,NegativeEmotions
93,95,alta,Alta: a detailed dreamer,5/22/1986,en,"I've just come back to the ""library"" from bein...",1MSA,NaN,1FSA > D,"D 4> Q, D 4> 2ISA",...,1.000000,0.000000,0.000000,0.000000,0.0,0.333333,1.000000,2.000000,0.000000,0.000000
305,311,alta,Alta: a detailed dreamer,2/17/1991,en,I'm in a theater where a rock band is about to...,"2ISA, 2IKA, 2FSA, 1ISA","AP 2ISA, HA D",D > 1FSA,"D 4> Q, D 4> Q, 2IKA 4> we, Q 4> D, D 4> Q",...,0.000000,0.000000,0.250000,0.000000,0.0,0.200000,0.250000,1.250000,0.000000,0.500000
352,358,alta,Alta: a detailed dreamer,12/15/1991,en,I've just gotten married to this wonderful guy...,"1ISA, 2ISA, 1FSA, 1IKA, 1MHA, 1MKA, 2ISA, 2ISA...","HA 1MSA, AP D, HA D, HA 2ISA, AP 1MSA, CO 1MSA","1MSA > D, D > Q, D > 1MKA, D > D, D > 1MSA, D ...","D 4> 1IKA, 1IKA 4> 1FSA, 1FSA 4> D, D 4> Q, D ...",...,0.666667,0.000000,0.222222,0.111111,0.0,0.461538,0.666667,1.111111,0.444444,0.500000
395,401,alta,Alta: a detailed dreamer,8/2/1992,en,"I am playing (sort of) with a Komodo dragon, a...",baby,"HA 1ISA, AP D, AP D","you PRP >, you PRP >, 2ISA > their, their PRP ...","D 4> 1ANI, you PRP 4>",...,0.000000,0.000000,0.000000,0.000000,0.0,0.750000,8.000000,2.000000,1.000000,0.666667
431,439,angie,Angie: age 18 & 20,5/9/1996,en,"I was in a place that I didn't recognize, but ...","1ISA, people, 1ISA, 1IFA",CO D,NaN,"1FSA 4> you, D 4> D, 2IKA 4> D, 1FSA 4> D, 1MS...",...,0.000000,0.000000,0.000000,0.250000,0.0,0.000000,0.000000,1.250000,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20915,32975,wedding,Wedding dreams,1995,en,I was to be married to some unknown man. We we...,"1MKA, 2ANI, 1ISA",NaN,NaN,"D 4> 1MKA, 1MKA 4> him, D 4> him, D 4> Q",...,1.000000,0.333333,0.500000,0.000000,0.0,0.000000,0.000000,1.333333,0.000000,0.000000
20917,32977,wedding,Wedding dreams,1995,en,"In all my wedding dreams, I'm getting married ...","2ISA, guy",CO D,D > PRP,"Q 4> D, D 4> Q, D 4> PRP, him PRP 4>, 1MSA 4> we",...,0.000000,0.000000,0.000000,0.000000,0.0,0.250000,0.500000,2.500000,0.000000,1.000000
20947,33013,west_coast_teens,West Coast teenage girls,"F, age 14",en,In the dream I was at the portables where math...,2ISA,NaN,D > Q,"D 4> D, D 4> Q, D 4> Q, D 4> Q",...,0.000000,0.000000,0.000000,0.000000,0.0,0.200000,1.000000,4.000000,1.000000,0.000000
20969,33036,west_coast_teens,West Coast teenage girls,"F, age 16",en,"I had a dream that I was at the R.E.M, concert...",1FSA,NaN,NaN,"1MSA 4> Q, 1MSA 4> Q",...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,2.000000,0.000000,0.000000


In [45]:
print(np.where(dreams['S/CIndex'] > 1.0))
dreams.query('`S/CIndex` > 1.0')

(array([  736,   753,   758,   761,   785,   799,   813,   893,  1017,
        1113,  1125,  1162,  1175,  1209,  1239,  1244,  1336,  1340,
        1343,  1414,  1465,  1482,  1502,  1628,  1674,  1755,  1898,
        1899,  2029,  2048,  2120,  2141,  2153,  2204,  2242,  2282,
        2287,  2301,  2308,  2342,  2485,  2497,  2503,  2510,  2556,
        2561,  2605,  2618,  2632,  2657,  2685,  2831,  2838,  2840,
        2855,  2856,  2866,  2870,  2880,  2902,  2903,  2942,  2962,
        2998,  3021,  3145,  3147,  3180,  3190,  3225,  3230,  3236,
        3245,  3265,  3298,  3308,  3390,  3485,  3506,  3507,  3526,
        3567,  3570,  3594,  3618,  3629,  3666,  3672,  3674,  3697,
        3721,  3777,  3794,  3815,  3821,  3858,  3860,  3862,  3864,
        3885,  3916,  3920,  3928,  3955,  3979,  4010,  4051,  4089,
        4163,  4168,  4187,  4200,  4218,  4230,  4264,  4285,  4364,
        4564,  4760,  4808,  5074,  5097,  5115,  5141,  5142,  5157,
        5281,  5535

,dream_id,dreamer,description,dream_date,dream_language,text_dream,characters_code,emotions_code,aggression_code,friendliness_code,...,Male,Animal,Friends,Family,Dead&Imaginary,Aggression/Friendliness,A/CIndex,F/CIndex,S/CIndex,NegativeEmotions
736,781,b,Barb Sanders,12/26/1988,en,I am swimming with a group of people in a swim...,"2ISA, 1MKA",AN D,"1MSA > D, D 1> Q","1MSA 4> D, D 4> him, D 4> him",...,1.0,0.0,0.500,0.000,0.0,0.400000,1.00,1.500000,2.000000,1.000000
753,798,b,Barb Sanders,5/13/1989,en,"Someone looked at my hair and said, ""Do you li...",1ISA,CO 1ISA,NaN,"D 4> Q, it PRP 4>, D 4> D",...,0.0,0.0,0.000,0.000,0.0,0.000000,0.00,3.000000,2.000000,1.000000
758,803,b,Barb Sanders,6/15/1989,en,"I am sexually excited and want to make love, b...",1MSA,"HA D, AP D",NaN,"D 4> Q, D 4> him, D 4> D",...,1.0,0.0,0.000,0.000,0.0,0.000000,0.00,3.000000,4.000000,0.500000
761,806,b,Barb Sanders,6/22/1989,en,I go through a huge store to the back room. I'...,"1MKA, 1ISA, 1IOA, 2ISA","AP 1MSA, AN D, SD 1MSA, HA 2ISA","D > Q, D > D, 1MSA > D, 2IKA > Q, D 1> Q","1MSA 4> D, 1MSA 4> D, D 4> 1IKA",...,1.0,0.0,0.250,0.000,0.0,0.571429,1.25,0.750000,1.250000,0.750000
785,830,b,Barb Sanders,9/25/1990,en,Howard wants to make love to me. I am paralyze...,1MSA,AP D,NaN,"1MKA 4> D, D 4> Q, D 4> him",...,1.0,0.0,0.000,0.000,0.0,0.000000,0.00,3.000000,4.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20093,25964,phil2,Phil 2: late 20s,3/2/1971,en,My wife Bonita and I were in a place like Shak...,"1FWA, 2ISA, 1ISA, 2ISA, 2ISA, 2MKA, girl, 1ISA","CO 1FSA, SD D","Q > Q, 1FSA > Q, 1FSA > Q, D > 1FSA, D > D, 1F...","2ISA 4> Q, 1FKA 4> Q, 1FSA 4> Q",...,0.5,0.0,0.125,0.125,0.0,1.000000,0.75,0.375000,1.375000,1.000000
20638,26567,tom,Tom: An outgoing man,?,en,I am at work and I am talking to two of the wa...,2FSA,"HA D, AP 2IKA, HA D",2ISA > Q,"D 4> Q, D 4> 2ISA, 2ISA 4> D",...,0.0,0.0,0.000,0.000,0.0,0.000000,1.00,3.000000,2.000000,0.333333
20674,26603,ucsc_women,"UCSC women, 1996",20/female,en,A big 4x4 truck was chasing after me on freewa...,"1ISA, 2ISA, 1IOA",NaN,"Q > D, D > Q, D > D","D 4> D, 1IOA 4> D, 1MSA 4> D, 1MSA 4> D",...,0.0,0.0,0.000,0.000,0.0,0.428571,1.00,1.333333,1.333333,0.000000
20946,33012,west_coast_teens,West Coast teenage girls,"F, age 14",en,In this dream I was walking around down all by...,1ISA,AP D,"1MSA > D, 1MSA > Q, D > D, D > Q",NaN,...,0.0,0.0,0.000,0.000,0.0,1.000000,4.00,0.000000,2.000000,1.000000
